# Red S6

In [1]:
import networkx as nx
import numpy as np
import pandas as pd

# Funciones

Principalmente para transformar la columna original de atributos `attrs` a un conjunto de columnas con los atributos de cada nodo.

## Variables globales

In [91]:
# Path al archivo de la red original
archivo = "../data_local/base_S6_hacky.graphml"

nodos_out = "../tabla_nodos.csv"
edges_out = "../tabla_edges.csv"

In [50]:
def get_attr(x, attr="labels"):
    """Regresa el valor de attr o None.
    
    x: dict, diccionario con los atributos en llave.
    attr: str, nombre del atributo
    """
    
    if attr in x.keys():
        return x[attr]
    else:
        return None

In [86]:
def n_list(x):
    """Regresa en número de elementos en x
    
    x: str, string representando a una lista
    """
    
    if x is None:
        return None
    else:
        x = eval(x)
        return(len(x))

In [3]:
# Leer red original en formato graphml
G = nx.read_graphml(archivo)

/home/x/.local/lib/python3.6/site-packages/networkx/readwrite/graphml.py:899: UserWarning: No key type for id name. Using string
  warnings.warn("No key type for id %s. Using string" % attr_id)
/home/x/.local/lib/python3.6/site-packages/networkx/readwrite/graphml.py:899: UserWarning: No key type for id procurementMethod. Using string
  warnings.warn("No key type for id %s. Using string" % attr_id)
/home/x/.local/lib/python3.6/site-packages/networkx/readwrite/graphml.py:899: UserWarning: No key type for id labels. Using string
  warnings.warn("No key type for id %s. Using string" % attr_id)
/home/x/.local/lib/python3.6/site-packages/networkx/readwrite/graphml.py:899: UserWarning: No key type for id id. Using string
  warnings.warn("No key type for id %s. Using string" % attr_id)
/home/x/.local/lib/python3.6/site-packages/networkx/readwrite/graphml.py:899: UserWarning: No key type for id title. Using string
  warnings.warn("No key type for id %s. Using string" % attr_id)
/home/x/.local/l

## Pandas

Usaremos pandas para clear la tabla con los nodos y sus atributos.

Básicamente pasamos los nodos en la red original `G` junto con sus atributos a un `pandas.DataFrame` y hacemos las transformaciones ahí mismo.

In [73]:
# Cargar dataframe
df_nodes = pd.DataFrame(G.nodes(data=True), columns=["nodo", "attrs"])
# Enlistar todos los atributos en el dataframe
key_nodes = np.array([list(row["attrs"].keys()) for i, row in df_nodes.iterrows()])

key_nodes = [x for y in key_nodes for x in y]

key_nodes = set(key_nodes)

In [74]:
print(f"Los atributos disponibles son:\n {key_nodes}")

Los atributos disponibles son:
 {'ocid', 'title', 'name', 'contracts', 'totalAmount', 'procurementMethod', 'labels', 'id'}


In [48]:
# Crear una columna por atributo disponible y
# al final borrar la columna de atributos
for attr in key_nodes:
    df_nodes[attr] = df_nodes["attrs"].apply(get_attr, attr=attr)

df_nodes = df_nodes.drop("attrs", axis=1)

### Transformaciones

Crear una variable con el número de subcontratos para cada contrato

In [63]:
df_nodes["nsubcontracts"] = df_nodes["contracts"].apply(n_list)

### Write out

In [70]:
# Guardar la tabla
df_nodes.to_csv(nodos_out, index=False)

# Aristas

Se crea una tabla con, para cada arista, los valores de cada atributo.

In [77]:
pd.DataFrame(G.edges(data=True))

,0,1,2
0,n0,n95307,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
1,n0,n95453,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
2,n0,n95460,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
3,n0,n95461,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
4,n0,n95462,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
...,...,...,...
999145,n95303,n460060,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."
999146,n95304,n460099,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."
999147,n95304,n460136,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."
999148,n95305,n460141,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."


In [78]:
# Cargar dataframe
df_edges = pd.DataFrame(G.edges(data=True), columns=["source", "target", "attrs"])
# Enlistar todos los atributos  de las aristas en el dataframe
key_edges = np.array([list(row["attrs"].keys()) for i, row in df_edges.iterrows()])
key_edges = [x for y in key_edges for x in y]
key_edges = set(key_edges)

In [79]:
df_edges

,source,target,attrs
0,n0,n95307,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
1,n0,n95453,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
2,n0,n95460,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
3,n0,n95461,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
4,n0,n95462,"{'label': 'BOUGHT', 'roles': '[""buyer""]'}"
...,...,...,...
999145,n95303,n460060,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."
999146,n95304,n460099,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."
999147,n95304,n460136,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."
999148,n95305,n460141,"{'label': 'PARTICIPATED', 'roles': '[""tenderer..."


In [80]:
key_edges

{'label', 'roles'}

In [81]:
print(f"Los atributos disponibles son:\n {key_edges}")

Los atributos disponibles son:
 {'label', 'roles'}


In [82]:
# Crear una columna por atributo disponible y
# al final borrar la columna de atributos
for attr in key_edges:
    df_edges[attr] = df_edges["attrs"].apply(get_attr, attr=attr)

df_edges = df_edges.drop("attrs", axis=1)

### Transformaciones

Agregamos variable con el número de posibles roles 

In [87]:
df_edges["nroles"] = df_edges["roles"].apply(n_list)

In [90]:
df_edges.roles.unique()

array(['["buyer"]', '["tenderer"]', '["tenderer","supplier"]'],
      dtype=object)

### Write out

In [92]:
# Guardar la tabla
df_edges.to_csv(edges_out, index=False)